In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import csv

leagues = [
    'https://www.transfermarkt.us/premier-league/startseite/wettbewerb/GB1',
    'https://www.transfermarkt.us/primera-division/startseite/wettbewerb/ES1',
    'https://www.transfermarkt.us/serie-a/startseite/wettbewerb/IT1',
    'https://www.transfermarkt.us/bundesliga/startseite/wettbewerb/L1',
    'https://www.transfermarkt.us/ligue-1/startseite/wettbewerb/FR1'
]

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

team_ids = set()
teams_file = 'teams.csv'


def get_teams(league_url, year):
    ext = f'/plus/?saison_id={year}'
    response = requests.get(league_url + ext, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    count = 0

    a_tags = soup.find_all('a', href=re.compile(r'/[^/]+/startseite/verein/\d+'))
    for tag in a_tags:
        href = tag.get('href')
        match = re.search(r'/(?P<name>[^/]+)/startseite/verein/(?P<id>\d+)', href)
        if match:
            name = match.group('name')
            team_id = match.group('id')
            if team_id not in team_ids:
                team_ids.add(team_id)
                writer.writerow([name, team_id])
                count += 1
    return count

with open(teams_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['name', 'id'])

    for year in range(2020, 2024):
        ext = f'/plus/?saison_id={year}'
        for league in leagues:
            match = re.search(r'/([^/]+)/startseite', league)
            if match:
                name = match.group(1)
            print(f'Getting teams for {name} in {year}')
            print(f'Found {get_teams(league, year)} teams')

In [10]:
from bs4 import BeautifulSoup
import requests
import re
import csv

player_file = 'players.csv'
teams_file = 'teams.csv'
team_url = "https://www.transfermarkt.us/team_name/kader/verein/id"
player_names = set()

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

def get_players(team_url, year):
    ext = f'/plus/0/galerie/0?saison_id={year}'
    response = requests.get(team_url + ext, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    a_tags = soup.find_all('a', href=re.compile(r'/[^/]+/profil/spieler/\d+'))
    count = 0

    for tag in a_tags:
        href = tag.get('href')
        match = re.search(r'/(?P<name>[^/]+)/profil/spieler/(?P<id>\d+)', href)
        if match:
            name = match.group('name')
            player_id = match.group('id')
            if player_id not in player_names:
                player_names.add(player_id)
                writer.writerow([name, player_id])
                count += 1
    return count


with open(teams_file, mode='r', newline='') as read_file:
    reader = csv.reader(read_file)
    next(reader)
    with open(player_file, mode='w', newline='') as write_file:
        writer = csv.writer(write_file)
        writer.writerow(['name', 'id'])
        for row in reader:
            team_name = row[0]
            team_id = row[1]
            for year in range(2020, 2024):
                print(f'Getting players for {team_name} in {year}')
                print(f'Found {get_players(team_url.replace('team_name', team_name).replace('id', team_id), year)} players')



Getting players for manchester-city in 2020
Found 36 players
Getting players for manchester-city in 2021
Found 12 players
Getting players for manchester-city in 2022
Found 11 players
Getting players for manchester-city in 2023
Found 5 players
Getting players for manchester-united in 2020
Found 39 players
Getting players for manchester-united in 2021
Found 13 players
Getting players for manchester-united in 2022
Found 15 players
Getting players for manchester-united in 2023
Found 8 players
Getting players for fc-liverpool in 2020
Found 43 players
Getting players for fc-liverpool in 2021
Found 16 players
Getting players for fc-liverpool in 2022
Found 10 players
Getting players for fc-liverpool in 2023
Found 4 players
Getting players for fc-chelsea in 2020
Found 37 players
Getting players for fc-chelsea in 2021
Found 18 players
Getting players for fc-chelsea in 2022
Found 18 players
Getting players for fc-chelsea in 2023
Found 12 players
Getting players for tottenham-hotspur in 2020
Found

KeyboardInterrupt: 